# Scraping Multiples pages of IMDB at a time to fetch top 1000 movies data.

Importing the libraries needed 


In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from time import sleep
from random import randint
import pandas as pd

Declaring the headers 


In [3]:
headers = {"Accept-Language": "en-US,en;q=0.5"}


Declaring the list of empty variables, So that we can append the data overall


In [4]:
movie_name=[]
year=[]
time=[]
movie_ratings=[]
genre_movies=[]
director_name=[]     
votes=[]
gross=[]


Creating an array of values and passing it in the url for dynamic webpages


In [5]:
pages=np.arange(1,1000,50)

The whole core of the script


In [6]:
for i in pages:
    page=requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start="+str(i)+"&ref_=adv_nxt")
    page.raise_for_status()
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.strip("()")
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        genre_movie=store.p.find("span", class_ = 'genre').text.replace('\n', '')
        genre_movies.append(genre_movie)
        
        ratings=store.find("div",class_="inline-block ratings-imdb-rating").text.replace('\n', '')
        movie_ratings.append(ratings)
        
        
        directors = store.find('p',class_="").a.text
        director_name.append(directors)
         
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)
        
       
        


         
 


    


In [9]:
movies_list=pd.DataFrame({"Movie":movie_name,"years":year,'Runtime':time,'Ratings':movie_ratings,'Genre':genre_movies,'Director':director_name,'votes':votes,'Gross':gross})
movies_list.head(2)

,Movie,years,Runtime,Ratings,Genre,Director,votes,Gross
0,The Dark Knight,2008,152 min,9.0,"Action, Crime, Drama",Christopher Nolan,"2,593,495",$534.86M
1,The Lord of the Rings: The Return of the King,2003,201 min,9.0,"Action, Adventure, Drama",Peter Jackson,"1,798,625",$377.85M


In [10]:
# #saving the data in excel format
movies_list.to_excel("Top 1000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movies_list.to_csv("Top 1000 IMDb movies.csv")
